In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import pandas as pd
import torch
from model import CustomRoberta
from preprocess_raw import process_dataframe
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer
from tqdm import tqdm

/home/tbrow51/.conda/envs/transformers/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_df = pd.read_csv("/opt/localdata/Data/bea/nlp/bmi550/project/full_tweet_dataset/chronic_pain_data.csv")
raw_df = raw_df.drop_duplicates(['tweet_id', 'text']).reset_index()
raw_df


,index,Unnamed: 0.1,Unnamed: 0,screen_name,created_at,sampled_at,id_str,text,tweet_id,location,keyword,collection_site
0,0,0,0,BonnieBarnes48,2010-09-19 23:10:18+00:00,2022-05-23 23:44:11+00:00,192711150,My neck is on FIRE from post herpetic neuralgi...,1528884526847500289,"Florida, USA",#chronicpain,Emory
1,1,1,1,Khaleesi_Magica,2012-05-05 16:49:14+00:00,2022-05-23 23:39:22+00:00,571898861,I cannot wait for my ouchie sweater from @amer...,1528883317269008389,In @JunoBelmont ❤️,#chronicpain,Emory
2,2,2,2,ThommyB,2007-12-15 09:15:43+00:00,2022-05-23 23:23:03+00:00,11191762,Fatigue: The Science of it #VIDEO Fibromyalgia...,1528879208058994688,"Detroit, MI",#chronicpain,Emory
3,3,3,3,AGholamrezaei,2015-06-23 21:58:43+00:00,2022-05-23 23:13:29+00:00,3343195649,Understanding the contribution of racially and...,1528876804370444288,"Sydney, AU",#chronicpain,Emory
4,4,4,4,my_chyp,2019-06-06 19:50:00+00:00,2022-05-23 23:00:59+00:00,1136721921880469504,Got the Monday Blues? Feeling stuck? You are n...,1528873658084737026,NaN,#chronicpain,Emory
...,...,...,...,...,...,...,...,...,...,...,...,...
92975,114069,92975,92975,genderebels,2021-01-14 16:34:47+00:00,2022-02-01 00:04:18+00:00,1349756827928580102,@cowboybunnies you’re describing chronic pain,1488302151898222604,18 | she/he/ze | latina butch,chronicpain,Emory
92976,114070,92976,92976,ANotoriousPIG,2013-03-03 19:38:59+00:00,2022-02-01 00:02:26+00:00,1239687583,@BrettButler1111 No judgment either way. I'd h...,1488301679674023945,In pain; in Maine 🦞,chronicpain,Emory
92977,114071,92977,92977,ValorPharmacy,2019-04-08 15:06:43+00:00,2022-02-01 00:01:13+00:00,1115269749003313152,Low-dose naltrexone can be used for chronic pa...,1488301373347217410,"Berkeley, CA",chronicpain,Emory
92978,114072,92978,92978,Mikeyvp,2009-03-18 21:00:47+00:00,2022-02-01 00:00:25+00:00,25153545,@tribjazz As a sufferer of chronic pain I won’...,1488301174008672257,Utah,chronicpain,Emory


In [3]:
proc_df = process_dataframe(raw_df)
proc_df

preprocessing dataframe


,index,Unnamed: 0.1,Unnamed: 0,screen_name,created_at,sampled_at,id_str,text,tweet_id,location,keyword,collection_site
0,0,0,0,BonnieBarnes48,2010-09-19 23:10:18+00:00,2022-05-23 23:44:11+00:00,192711150,my neck is on fire <allcaps> from post herpeti...,1528884526847500289,"Florida, USA",#chronicpain,Emory
1,1,1,1,Khaleesi_Magica,2012-05-05 16:49:14+00:00,2022-05-23 23:39:22+00:00,571898861,i cannot wait for my ouchie sweater from <user...,1528883317269008389,In @JunoBelmont ❤️,#chronicpain,Emory
2,2,2,2,ThommyB,2007-12-15 09:15:43+00:00,2022-05-23 23:23:03+00:00,11191762,fatigue: the science of it video fibromyalgi...,1528879208058994688,"Detroit, MI",#chronicpain,Emory
3,3,3,3,AGholamrezaei,2015-06-23 21:58:43+00:00,2022-05-23 23:13:29+00:00,3343195649,understanding the contribution of racially and...,1528876804370444288,"Sydney, AU",#chronicpain,Emory
4,4,4,4,my_chyp,2019-06-06 19:50:00+00:00,2022-05-23 23:00:59+00:00,1136721921880469504,got the monday blues? feeling stuck? you are n...,1528873658084737026,NaN,#chronicpain,Emory
...,...,...,...,...,...,...,...,...,...,...,...,...
92975,114069,92975,92975,genderebels,2021-01-14 16:34:47+00:00,2022-02-01 00:04:18+00:00,1349756827928580102,<user> you’re describing chronic pain,1488302151898222604,18 | she/he/ze | latina butch,chronicpain,Emory
92976,114070,92976,92976,ANotoriousPIG,2013-03-03 19:38:59+00:00,2022-02-01 00:02:26+00:00,1239687583,<user> no judgment either way. i'd hate make s...,1488301679674023945,In pain; in Maine 🦞,chronicpain,Emory
92977,114071,92977,92977,ValorPharmacy,2019-04-08 15:06:43+00:00,2022-02-01 00:01:13+00:00,1115269749003313152,low-dose naltrexone can be used for chronic pa...,1488301373347217410,"Berkeley, CA",chronicpain,Emory
92978,114072,92978,92978,Mikeyvp,2009-03-18 21:00:47+00:00,2022-02-01 00:00:25+00:00,25153545,<user> as a sufferer of chronic pain i won’t e...,1488301174008672257,Utah,chronicpain,Emory


In [4]:
init_proc_save_path = "/opt/localdata/Data/bea/nlp/bmi550/project/full_tweet_dataset/chronic_pain_proc_data.csv"

proc_df.to_csv(init_proc_save_path, index=False)

In [4]:
class TweetDataset(Dataset):
    """
    class is very closely based on the huggingface tutorial implementation
    """
    def __init__(self, dataframe, tokenizer, max_len, id_col: str = 'tweet_id',
                 text_col: str = 'text', target_col: str = 'class_label'):
        self.tokenizer = tokenizer
        # self.data = dataframe
        self.tweet_id_list = list(dataframe[id_col])
        self.text_list = list(dataframe[text_col])
        # self.label_list = list(dataframe[target_col])
        self.max_len = max_len

    def __len__(self):
        # get length of dataset (required for dataloader)
        return len(self.text_list)

    def __getitem__(self, idx):
        # extract text
        text = str(self.text_list[idx])

        # tokenize text
        encoded_text = self.tokenizer.encode_plus(
            text,
            # add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )

        # unpack encoded text
        ids = encoded_text['input_ids']
        attention_mask = encoded_text['attention_mask']
        token_type_ids = encoded_text["token_type_ids"]

        # wrap outputs in dict
        out_dict = {
            'tweet_id_list': self.tweet_id_list[idx],
            'id_tensor': torch.tensor(ids, dtype=torch.long),
            'mask_tensor': torch.tensor(attention_mask, dtype=torch.long),
            'token_type_tensor': torch.tensor(token_type_ids, dtype=torch.long),
        }

        return out_dict
    

def get_dataloader(dataset, batch_size, shuffle: bool = True,
                   pin_memory: bool = True, num_workers: int = 0,
                   prefetch_factor: int or None = None):
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        pin_memory=pin_memory,
        num_workers=num_workers,
        prefetch_factor=prefetch_factor
    )
    return dataloader

In [5]:
# load roberta base as a tokenizer
tokenizer = RobertaTokenizer.from_pretrained(
    'roberta-base', 
    truncation=True, 
    do_lower_case=True
)

max_len = 256
batch_size = 128

test_ds = TweetDataset(proc_df, tokenizer, max_len)
test_loader = get_dataloader(test_ds, batch_size)

In [6]:
# seed torch operations
torch.manual_seed(13)

In [7]:
# get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"using {device} device...")

# build the model and send it to the gpu
model = CustomRoberta(0.3, 1)
model.to(device)

# load data
model_save_path = "/opt/localdata/Data/bea/nlp/bmi550/project/Chronic_Pain_Sentiment_NLP/models/chronic_pain_self_report/final_model/best_model_params.pth"
model.load_state_dict(torch.load(model_save_path))

using cuda device...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [9]:
def infer_model(model, device, test_loader):
    running_loss = 0.0
    
    tweet_id_list = []
    pred_list = []
    label_list = []

    phase_size = len(test_loader)

    # iterate over data in current phase loader
    with tqdm(test_loader, unit="batch", total=phase_size) as epoch_iter:
        for batch, data in enumerate(epoch_iter):
            # unpack data dict
            batch_id_list = data['tweet_id_list']
            id_tensor = data['id_tensor'].to(device)
            mask_tensor = data['mask_tensor'].to(device)
            token_type_tensor = data['token_type_tensor'].to(device)
            # label_tensor = data['label_tensor'].to(device)

            # forward
            # track history if only in train
            with torch.set_grad_enabled(False):
                outputs = model(
                    id_tensor,
                    mask_tensor,
                    token_type_tensor
                )
                preds = torch.squeeze(outputs)
                
                tweet_id_list += batch_id_list
                pred_list += list(preds.detach().cpu()) #.numpy())
        
    return tweet_id_list, pred_list

In [10]:
tweet_id_list, pred_list = infer_model(model, device, test_loader)

100%|██████████| 727/727 [05:32<00:00,  2.19batch/s]


In [15]:
tweet_id_list = [t.item() for t in tweet_id_list]
tweet_id_list

[1524694260594978818,
 1520359057395265538,
 1527871681943027713,
 1518660138994872321,
 1497338116306264068,
 1504107198124802048,
 1511534823168430081,
 1489971607112241156,
 1515680551654465537,
 1504481416205127683,
 1513885764542734348,
 1496341563714285569,
 1519715461406986240,
 1501938300289880066,
 1503476222772187136,
 1489145984336777220,
 1498214932126568451,
 1492500911511506948,
 1509570372173283330,
 1509921866588295170,
 1526599174170595328,
 1491344117733142531,
 1518778927820410880,
 1493182122697256961,
 1527749177912045569,
 1513007953053376518,
 1518314363941232643,
 1519077447815344128,
 1512381790933209089,
 1511839572212211727,
 1527359900661485573,
 1518573415187337217,
 1515332425299243012,
 1507483721452171265,
 1500810342523408385,
 1512485748112433154,
 1518996090233798657,
 1505822484599390210,
 1502352055293427723,
 1528252648868159488,
 1515144479388315651,
 1494448709320519692,
 1517365486304038914,
 1510987257934663681,
 1508421181388607499,
 152335191

In [16]:
pred_list = [p.item() for p in pred_list]
pred_list

[0.9542413949966431,
 0.9776002764701843,
 0.9735084176063538,
 0.6624374389648438,
 0.011198598891496658,
 0.6669341921806335,
 0.0059403423219919205,
 0.9771851897239685,
 0.9818072319030762,
 0.9781315922737122,
 0.9435193538665771,
 0.9721031785011292,
 0.43470820784568787,
 0.6986489295959473,
 0.9750900268554688,
 0.9814460873603821,
 0.00553766917437315,
 0.018627610057592392,
 0.9777964949607849,
 0.970127284526825,
 0.9061617255210876,
 0.00628195283934474,
 0.9464220404624939,
 0.9190411567687988,
 0.016721943393349648,
 0.9766238331794739,
 0.01095594372600317,
 0.006822533439844847,
 0.004801588132977486,
 0.014180609956383705,
 0.003409235505387187,
 0.978203296661377,
 0.981802761554718,
 0.009077545255422592,
 0.009290207177400589,
 0.006379401311278343,
 0.015296181663870811,
 0.9748920798301697,
 0.007029975764453411,
 0.004456766415387392,
 0.9604038596153259,
 0.20486289262771606,
 0.9661739468574524,
 0.043797317892313004,
 0.004308824893087149,
 0.9840050935745239,

In [17]:
pred_dict = dict(zip(tweet_id_list, pred_list))
pred_dict

{1524694260594978818: 0.9542413949966431,
 1520359057395265538: 0.9776002764701843,
 1527871681943027713: 0.9735084176063538,
 1518660138994872321: 0.6624374389648438,
 1497338116306264068: 0.011198598891496658,
 1504107198124802048: 0.6669341921806335,
 1511534823168430081: 0.0059403423219919205,
 1489971607112241156: 0.9771851897239685,
 1515680551654465537: 0.9818072319030762,
 1504481416205127683: 0.9781315922737122,
 1513885764542734348: 0.9435193538665771,
 1496341563714285569: 0.9721031785011292,
 1519715461406986240: 0.43470820784568787,
 1501938300289880066: 0.6986489295959473,
 1503476222772187136: 0.9750900268554688,
 1489145984336777220: 0.9814460873603821,
 1498214932126568451: 0.00553766917437315,
 1492500911511506948: 0.018627610057592392,
 1509570372173283330: 0.9777964949607849,
 1509921866588295170: 0.970127284526825,
 1526599174170595328: 0.9061617255210876,
 1491344117733142531: 0.00628195283934474,
 1518778927820410880: 0.9464220404624939,
 1493182122697256961: 0.9

In [18]:
proc_df.loc[:, 'prediction'] = proc_df.loc[:, 'tweet_id'].map(pred_dict)
proc_df

,index,Unnamed: 0.1,Unnamed: 0,screen_name,created_at,sampled_at,id_str,text,tweet_id,location,keyword,collection_site,prediction
0,0,0,0,BonnieBarnes48,2010-09-19 23:10:18+00:00,2022-05-23 23:44:11+00:00,192711150,my neck is on fire <allcaps> from post herpeti...,1528884526847500289,"Florida, USA",#chronicpain,Emory,0.981875
1,1,1,1,Khaleesi_Magica,2012-05-05 16:49:14+00:00,2022-05-23 23:39:22+00:00,571898861,i cannot wait for my ouchie sweater from <user...,1528883317269008389,In @JunoBelmont ❤️,#chronicpain,Emory,0.954881
2,2,2,2,ThommyB,2007-12-15 09:15:43+00:00,2022-05-23 23:23:03+00:00,11191762,fatigue: the science of it video fibromyalgi...,1528879208058994688,"Detroit, MI",#chronicpain,Emory,0.006235
3,3,3,3,AGholamrezaei,2015-06-23 21:58:43+00:00,2022-05-23 23:13:29+00:00,3343195649,understanding the contribution of racially and...,1528876804370444288,"Sydney, AU",#chronicpain,Emory,0.005532
4,4,4,4,my_chyp,2019-06-06 19:50:00+00:00,2022-05-23 23:00:59+00:00,1136721921880469504,got the monday blues? feeling stuck? you are n...,1528873658084737026,NaN,#chronicpain,Emory,0.006020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92975,114069,92975,92975,genderebels,2021-01-14 16:34:47+00:00,2022-02-01 00:04:18+00:00,1349756827928580102,<user> you’re describing chronic pain,1488302151898222604,18 | she/he/ze | latina butch,chronicpain,Emory,0.008860
92976,114070,92976,92976,ANotoriousPIG,2013-03-03 19:38:59+00:00,2022-02-01 00:02:26+00:00,1239687583,<user> no judgment either way. i'd hate make s...,1488301679674023945,In pain; in Maine 🦞,chronicpain,Emory,0.969234
92977,114071,92977,92977,ValorPharmacy,2019-04-08 15:06:43+00:00,2022-02-01 00:01:13+00:00,1115269749003313152,low-dose naltrexone can be used for chronic pa...,1488301373347217410,"Berkeley, CA",chronicpain,Emory,0.004278
92978,114072,92978,92978,Mikeyvp,2009-03-18 21:00:47+00:00,2022-02-01 00:00:25+00:00,25153545,<user> as a sufferer of chronic pain i won’t e...,1488301174008672257,Utah,chronicpain,Emory,0.984626


In [25]:
thresh = 0.95

proc_df.loc[:, 'binary_pred'] = proc_df.loc[:, 'prediction'].apply(lambda x: x > thresh)
proc_df.binary_pred.value_counts(dropna=False)

binary_pred
False    59089
True     33891
Name: count, dtype: int64

In [26]:
refined_df = proc_df.loc[proc_df['binary_pred'] == True, :]
refined_df

,index,Unnamed: 0.1,Unnamed: 0,screen_name,created_at,sampled_at,id_str,text,tweet_id,location,keyword,collection_site,prediction,binary_pred
0,0,0,0,BonnieBarnes48,2010-09-19 23:10:18+00:00,2022-05-23 23:44:11+00:00,192711150,my neck is on fire <allcaps> from post herpeti...,1528884526847500289,"Florida, USA",#chronicpain,Emory,0.981875,True
1,1,1,1,Khaleesi_Magica,2012-05-05 16:49:14+00:00,2022-05-23 23:39:22+00:00,571898861,i cannot wait for my ouchie sweater from <user...,1528883317269008389,In @JunoBelmont ❤️,#chronicpain,Emory,0.954881,True
6,6,6,6,PaulMoseley1874,2016-04-17 01:39:33+00:00,2022-05-23 22:25:28+00:00,721513397079183360,fingers crossed i have a better night than las...,1528864718999683072,"West Midlands, England",#chronicpain,Emory,0.978658,True
7,7,7,7,EmRenae1218,2011-12-19 23:33:46+00:00,2022-05-23 22:17:47+00:00,441305209,the guilt and grief i put myself through just ...,1528862783760400385,NaN,#chronicpain,Emory,0.966581,True
10,10,10,10,mikes_recovery,2016-12-11 10:03:21+00:00,2022-05-23 21:33:52+00:00,807888491518443520,i can't remember the last time i had such a ba...,1528851735028330496,"Thurawal country, Sydney, NSW",#chronicpain,Emory,0.977834,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92968,114062,92968,92968,GimpGirl64,2011-08-07 15:05:21+00:00,2022-02-01 00:18:59+00:00,350302850,"<user> i’m disabled, too. living with chronic ...",1488305842986360837,"Miami, Florida",chronicpain,Emory,0.969012,True
92971,114065,92971,92971,Wittywhitcomb,2010-08-29 21:35:43+00:00,2022-02-01 00:15:40+00:00,184550771,"i tell them about my pain, my electrical shock...",1488305011339587587,"Longmont, CO",chronicpain,Emory,0.978784,True
92972,114066,92972,92972,SkinlessGestate,2018-02-15 15:33:09+00:00,2022-02-01 00:14:52+00:00,964160658383212546,<user> as someone who has dealt with chronic p...,1488304809531047940,Detroit metro area,chronicpain,Emory,0.964581,True
92976,114070,92976,92976,ANotoriousPIG,2013-03-03 19:38:59+00:00,2022-02-01 00:02:26+00:00,1239687583,<user> no judgment either way. i'd hate make s...,1488301679674023945,In pain; in Maine 🦞,chronicpain,Emory,0.969234,True


In [28]:
for i, data in refined_df.sample(100).iterrows():
    print(f"\n{i}")
    print(data.text)


20084
<hashtag>  n e i svoid how can i manage sudden change in weather! like from living in a <number> to <number> deg celcius to minus and single digit temperatures! any advice on how to manage the pain in snowy winters and other symptoms that can pop up unexpectedly? <hashtag> fibromialgia <hashtag>  fibromyalgia <hashtag> chronicpain

71598
<user> i know i shouldn’t respond emotionally, but i’m getting pissed of by comments like the one concerning keith lee. the man is wresting only a few months after dealing with severe long covid <allcaps>. 

i had mild covid <allcaps>, at the worst, in january and my inflammatory issues / chronic pain <number> / <number>

8524
so excited to see where my fitness can go without <hashtag> endometriosis <hashtag> chronicpain holding me back!

44999
<user> <user> i just don't like <number> story homes, simply because of my long time chronic pain issues. my current home is <number>sf. i'd be happy if there was maybe one more bedroom, so we can all hav

In [29]:
pred_proc_save_path = "/opt/localdata/Data/bea/nlp/bmi550/project/full_tweet_dataset/chronic_pain_pred_data.csv"

proc_df.to_csv(pred_proc_save_path, index=False)

In [30]:
refined_save_path = "/opt/localdata/Data/bea/nlp/bmi550/project/full_tweet_dataset/chronic_pain_refined_data.csv"

refined_df.to_csv(refined_save_path, index=False)